Downloading necessary packages

In [ ]:
# !pip install langchain
# !pip install google-generativeai
# !pip install langchain_experimental
# !pip install pymysql
# !pip install sentence-transformers
# !pip install chromadb

Setting up the Google Palm LLM model using api key

In [22]:
# from langchain_community.llms import GooglePalm
# from google.generativeai import GooglePalm
# from langchain.llm import GooglePalm
import os
from dotenv import load_dotenv
load_dotenv()

# llm = GooglePalm(google_api_key=os.environ['GOOGLE_API_KEY'], temperature=0.1)

True

In [23]:
from langchain_google_genai import GoogleGenerativeAI
import os
llm = GoogleGenerativeAI(model="models/text-bison-001", google_api_key=os.environ['GOOGLE_API_KEY'])

Setting up the database to communicate with it

In [24]:
# for databases
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
import os

db_user = "root"
db_host = "localhost"
db_name = "atliq_tshirts"

db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{os.environ['db_password']}@{db_host}/{db_name}",sample_rows_in_table_info=3)
# print(db.table_info)

In [25]:
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose = True)
qns1 = db_chain.run("How many t-shirts do we have left for nike in extra small size and white color?")
qns2 = db_chain.run("How much is the price of the inventory for all small size t-shirts?")
qns3 = db_chain.run("SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'")
qns4 = db_chain.run("SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'")
qns5 = db_chain.run("SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'")



> Entering new SQLDatabaseChain chain...
How many t-shirts do we have left for nike in extra small size and white color?
SQLQuery:SELECT stock_quantity FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'
SQLResult: [(58,)]
Answer:58
> Finished chain.


> Entering new SQLDatabaseChain chain...
How much is the price of the inventory for all small size t-shirts?
SQLQuery:SELECT SUM(price) FROM t_shirts WHERE size = 'S'
SQLResult: [(Decimal('263'),)]
Answer:263
> Finished chain.


> Entering new SQLDatabaseChain chain...
SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'
SQLQuery:SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'
SQLResult: [(Decimal('19785'),)]
Answer:19785
> Finished chain.


> Entering new SQLDatabaseChain chain...
SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLQuery:SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLResult: [(Decimal('28305'),)]
Answer:28305
> Finished chain.

In [27]:
few_shots = [
    {'Question': "How many t-shirts do we have left for Nike in XS size and white color?",
     'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
     'SQLResult': "Result of the SQL query",
     'Answer': "58 t-shirts have been left for Nike in XS size and white color in the store"},
    {'Question': "How much is the total price of the inventory for all S-size t-shirts?",
     'SQLQuery': "SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'",
     'SQLResult': "Result of the SQL query",
     'Answer': "The total price of the inventory for all S-size t-shirts is twenty-two thousand two hundred ninety-two"},
    {'Question': "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue will our store generate (post discounts)?",
     'SQLQuery': """SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """,
     'SQLResult': "Result of the SQL query",
     'Answer': "The store will generate a revenue of sixteen thousand seven hundred twenty-five point four after applying discounts to all the Levi’s T-shirts"},
     {'Question': "If we have to sell all the Levi’s T-shirts today. How much revenue will our store generate without discount?",
      'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
      'SQLResult': "Result of the SQL query",
      'Answer': "The store will generate a revenue of seventeen thousand four hundred sixty-two without any discounts for all the Levi’s T-shirts"},
    {'Question': "How many white color Levi's shirts do I have?",
     'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
     'SQLResult': "Result of the SQL query",
     'Answer': "There are 290 white color Levi's shirts in the store"},
    {'Question': "How much sales amount will be generated if we sell all large size t-shirts today in Nike brand after discounts?",
     'SQLQuery': """SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Nike' and size="L"
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """,
     'SQLResult': "Result of the SQL query",
     'Answer': "The sales amount generated after selling all large size t-shirts today in Nike brand, post discounts, will be 290"} 
]


In [28]:
# for converting to word embeddings
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')

# creating a l=blob for our vector database
to_vectorise = [" ".join(temp.values()) for temp in few_shots]

In [29]:
# creating a vector database
from langchain.vectorstores import Chroma

vector_db = Chroma.from_texts(to_vectorise, embeddings, metadatas=few_shots)

In [30]:
# to be able to select similar vector from a given database
from langchain.prompts import SemanticSimilarityExampleSelector

example_selector = SemanticSimilarityExampleSelector(vectorstore=vector_db, k=2,)

In [31]:
# for custom inmstructions
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt

# print(_mysql_prompt)
# print(PROMPT_SUFFIX)
# overall our query will be in between the prefix and suffix

In [32]:
from langchain.prompts.prompt import PromptTemplate

prompt_skeleton = PromptTemplate(input_variables=["Question","SQLQuery","SQLResult","Answer"],
                                 template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}")

In [33]:
from langchain.prompts import FewShotPromptTemplate

few_shot_template = FewShotPromptTemplate(example_selector=example_selector, example_prompt=prompt_skeleton,
                                          prefix=_mysql_prompt,suffix=PROMPT_SUFFIX,
                                          input_variables=["input","table_info","top_k"])

In [34]:
new_db_chain = SQLDatabaseChain(llm=llm, database=db,prompt=few_shot_template)

/Users/omdeshmukh/Downloads/SemVI/NLDBQuery/.venv/lib/python3.9/site-packages/langchain_experimental/sql/base.py:76: UserWarning: Directly instantiating an SQLDatabaseChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [45]:
answer = new_db_chain.run("What are the total number brands of tshirts in database")
question = "What are the total number of tshirts in database"
# final_answer_prompt = PromptTemplate.from_template(
#     "Convert the question {question} and its answer {answer} pair to good english."
# )
s = f"Convert the question {question} and its answer {answer} pair to good answer in english. Only return you final answer, nothing else"
# final_answer_prompt.format(question = question, answer = answer)
llm.invoke(s)

'There are 4 brands of t-shirts in the database.'